In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import chisquare

Here we consider a truncated poisson distribution (Erlang System).
$$
P(i) = c \cdot \frac{A^i}{i!} \;, \;\; i = 0,\dots,m
$$
It's often used to simulate the probability of the number of busy lines in a communication system, train tracks and highways.

We'll visualize the probability masses for $m=10$ states to get an idea of what it looks like.

In [ ]:
def truncated_Poisson_distr(A, m):
    """Truncated poisson distribution (Erlang System)"""
    distr = {}
    var = np.arange(0, m + 1)
    for i in var:
        distr[i] = A**i / np.math.factorial(i)

    tot = np.sum(list(distr.values()))
    distr = {k: v / tot for k, v in distr.items()}  # Normalization
    return distr

# Sample the truncated poisson distribution
A = 8
m = 10
distr = truncated_Poisson_distr(A, m)

# Visualize the distribution
fig, ax = plt.subplots(1, 1, figsize=(20, 4))
ax.bar(x=[str(t) for t in distr.keys()], height=list(distr.values()), color="C0")
ax.set(
    xlabel="Discrete variable",
    ylabel="Probability",
    title=rf"Truncated Poisson distribution ($A$={A}, $m$={m})",
)
plt.xticks(rotation=90)
plt.show()

And we want to sample from it using Markov Chain Monte Carlo.

1. First we choose a proposal distribution $q$ which covers the density of the target distribution. 
$$
q(j|i) = \left\{
    \begin{array}{ll}
        0.5\;\; \text{if}\:\:\:|j-i| =1\:\:\:\: \text{or}\:\:\:\:\: |j-i|=m \\
        0 \;\;\;\;\text{otherwise}
    \end{array}
\right.
$$
Note that it is symmetric, $q(j|i) = q(i|j)$. When sampling from $q$ the candidate state $j$ can only be $i-j$ or $i+j$.
The proposal distribution is easy to sample from, and uniformly covers the target density.

But we need to be picky in terms of which samples we decide to keep. Or else we'll just end up with a uniform distribution that looks nothing like the target distribution...

2. We pick a suitable acceptance probability $\alpha$. 
$$
\alpha = \min(1, \frac{P(j)q(j|i)}{P(i)q(i|j)}) = \min(1, \frac{P(j)}{P(i)}) = \min(1, \frac{A^{j-i}\:i!}{j!})
$$
If we think about we've defined a ratio that can only ever get below 1 if the state changes in the positive direction $i+j$.
And it ensures that the sampled densities stay proportional to the given state of the target distribution.

In [ ]:
def sample_q(i, m):
    """Proposal distribution (one step forward or one step backward)"""
    if np.random.uniform() < 0.5:
        j = i - 1
        if i == 0:
            j = m
    else:
        j = i + 1
        if i == m:
            j = 0
    return j

def MH_algorithm_1(A, m, s_0, N_iter):
    """Markov chain algorithm (Only accept cases stepping forward that also satify the 20%-50% ratio)"""
    counter_accept = 0
    traces = []

    i = s_0
    for n in range(N_iter):

        j = sample_q(i, m)

        ratio = A ** (j - i) * np.math.factorial(i) / np.math.factorial(j)
        alpha = min(1, ratio)

        if np.random.uniform() < alpha:  # Accept
            i = j
            counter_accept += 1

        # Else, reject and i stays i.
        traces.append(i)

    print(f"Proposal accepted {100 * counter_accept / N_iter:2.3f}% of the times")
    return np.array(traces)

In [ ]:
N_iter = 50000
s_0 = 2
trace = MH_algorithm_1(A, m, s_0, N_iter)